## Calculate the ultimate delegate and cutoff texts beyond that

So we do not get spurious words in the resumption marked as delegates

In [382]:
from republic.model.republic_phrase_model import resolution_phrase_model
resolution_phrase_model[-3:]
resumption = resolution_phrase_model[-3:]

[{'phrase': 'BY refumptie gedelibereert zynde',
  'label': 'resolution_opening',
  'max_offset': 4},
 {'phrase': 'DE Resolutien, gisteren genomen',
  'label': 'resolution_summarized',
  'max_offset': 4,
  'variants': ['DE Resolutien, eergisteren genomen',
   'DE Resolutien voorleede ']},
 {'phrase': 'zyn gelesen en gerefumeert', 'label': 'resolution_summarized'}]

In [392]:
yesterdays = ['gisteren', 'eergisteren', 'voorleden donderdag',
              'voorleden vrijdagh', 'voorlede saterdagh']
searchstring = "DE Resolutien {g} ge-"
for g in yesterdays:
    samenv.append(searchstring.format(g=g))
resumption[1]["variants"].extend(samenv)

In [393]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

# highger matching thresholds for higher quality OCR/HTR (higher precision, recall should be good anyway)
# lower matching thresholds for lower quality OCR/HTR (higher recall, as that's the main problem)
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# initialize a new searcher instance with the config
resumption_searcher = FuzzyPhraseSearcher(config)

# create a list of domain keywords and phrases
#variants = [{'phrase': k, 'variants': v['variants']} for k, v in runner1715.all_matched.items()]
phrase = "DE Reslolutien, gisteren genomen, zijn gelesen ende geresumeert, gelijck"

# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(resumption)
# phrase_model.add_variants(variants)
resumption_searcher.index_phrase_model(phrase_model=phrase_model)

In [395]:
from scipy.special import softmax
from numpy import argmax
def score_match(match):
    result  = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_distance_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result



In [396]:
res

[PhraseMatch(phrase: "DE Resolutien, gisteren genomen", variant: "DE Resolutien, gisteren genomen",string: "DE Resolutien, gisteren genomen", offset: 342)]

In [ ]:
for T in runner1730.searchobs:
    ob = runner1730.searchobs[T]
    res = resumption_searcher.find_matches(ob.text, include_variants=True, use_word_boundaries=False)
    if res:
        softscore = softmax([score_match(match) for match in res])
        i = argmax(softscore)
        res[i].offset

now we try to truncate the text after the last delegate. The delegates in the attendance lists appear in a fixed order of provinces. As there were a rotating, but limited number of delegates for the provinces at any given time, there is an even smaller number of delegates who appear as the last delegate in the attendancelist. For the 1715 year for instance the most common last delegate is Van der Lelij, who appeared 254 times, almost 32 times the 8 appearence of the next frequent delegate (du Tour). 

We check the attendance lists for inconsistencies. Even if the delegate for the last province (Friesland - English *Frysia*) was not present, or if the matcher missed him this will still truncate spurious delegate markers.


In [20]:
from collections import Counter
lastfragments = []
lastpatterns = []
for ob in runner1730.searchobs:
    lastknownfragment = ''
    lastpattern = ''
    fragments = runner1730.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        f = item.get('delegate_name') or ''
        p = item.get('pattern') or ''
        if  f != '':
            lastknownfragment = item['delegate_name']
            lastfragments.append(lastknownfragment)
            lastpattern = p
            lastpatterns.append(lastpattern)
            break


In [21]:
# last marked delegates:
c = Counter(lastfragments)
# c.most_common():
#     print(i[0], i[1])
pd.DataFrame(c.most_common(), columns=['name', 'number'], )


,name,number
0,Tamminga van Alberda,81
1,Berghuis,77
2,van Lintelo,29
3,Queisen,27
4,Clant,24
5,van der Waeyen,12
6,van Isselmuden tot Zwollingerkamp,10
7,van Welderen,6
8,van Sloterdijck,6
9,Baert,6


In [23]:
for pers in Counter(lastfragments).most_common(10):
    print(pers, [goodvariants[d] for d in goodvariants if goodvariants[d]['match_name'].lower() == pers[0].lower()])
    #print(pers, [runner.all_matched[d] for d in runner.all_matched if runner.all_matched[d]['name'].lower() == pers[0].lower()])

NameError: name 'goodvariants' is not defined

In [22]:
common_last_delegates = [pers[0] for pers in Counter(lastfragments).most_common(10)]
common_last_delegates

['Tamminga van Alberda',
 'Berghuis',
 'van Lintelo',
 'Queisen',
 'Clant',
 'van der Waeyen',
 'van Isselmuden tot Zwollingerkamp',
 'van Welderen',
 'van Sloterdijck',
 'Baert']

- TODO: search after last delegate for common other last delegates
- TODO: truncate text and spans after last known delegate.